In [1]:
import maboss
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import shutil
import sys
import yaml
import numpy as np
sys.path.append("/Users/emilieyu/endotehelial-masboss")

ipylab module is not installed, menus and toolbar are disabled.


In [ ]:
from boolean_models.analysis import (
    compute_delta,
    classify_phenotype,
    save_df_to_csv
)
from boolean_models.scripts.run_perturbations import run_perturbations


In [ ]:
PROJECT_ROOT = Path("/Users/emilieyu/endotehelial-masboss/")
CONFIG_PATH = PROJECT_ROOT / "config" / "rho_sim_config.yaml"

with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

# Resolve Paths
MODELS_BND = PROJECT_ROOT / config['paths']['model_bnd']

model_cfg1 = PROJECT_ROOT / 'data' / 'models' / 'rho_v1.cfg'
model_cfg2 = PROJECT_ROOT / 'data' / 'models' / 'rho_v2.cfg'



In [ ]:
base_model = maboss.load(str(MODELS_BND), str(model_cfg1))



In [5]:
perb_df = run_perturbations()

DEUBG: Loaded base MaBoSS model
DEUBG: Built result directories.
DEBUG: Running scenario: WT
DEBUG: Computing delta = RhoC - RhoA
DEBUG: Running scenario: DSP
DEBUG: Computing delta = RhoC - RhoA
DEBUG: Running scenario: TJP1
DEBUG: Computing delta = RhoC - RhoA
DEBUG: Running scenario: JCAD
DEBUG: Computing delta = RhoC - RhoA
DEBUG: Running scenario: DSP_JCAD
DEBUG: Computing delta = RhoC - RhoA
DEBUG: Running scenario: TJP1_JCAD
DEBUG: Computing delta = RhoC - RhoA
DEBUG: All simulations completed successfully
DEBUG: File perturbation_timeseries.csv successfully written to directory: /Users/emilieyu/endotehelial-masboss/results/boolean_models/perturbation_sim


In [6]:
perb_df.tail(5)



,t,DSP,TJP1,JCAD,RhoA,RhoC,perturbation,delta,phenotype
235,3.5,0.497400,0.0,0.0,0.645618,0.417337,TJP1_JCAD,-0.228281,Normal
236,3.6,0.497400,0.0,0.0,0.645260,0.413584,TJP1_JCAD,-0.231676,Normal
237,3.7,0.497400,0.0,0.0,0.645279,0.412785,TJP1_JCAD,-0.232494,Normal
238,3.8,0.497401,0.0,0.0,0.646455,0.412741,TJP1_JCAD,-0.233714,Normal
239,3.9,0.497400,0.0,0.0,0.645544,0.412763,TJP1_JCAD,-0.232781,Normal


In [ ]:
ss_df = perb_df.groupby('perturbation')[perb_df['t'] == ]
ss_df

,t,DSP,TJP1,JCAD,RhoA,RhoC,perturbation,delta,phenotype
39,3.9,0.497999,0.505101,0.496301,0.597731,0.597317,WT,-0.000414,Normal
79,3.9,0.000000,0.505400,0.499900,0.388387,0.689580,DSP,0.301193,Hyper
119,3.9,0.500300,0.000000,0.499800,0.679798,0.398290,TJP1,-0.281508,Failed
159,3.9,0.494200,0.497100,0.000000,0.569283,0.570042,JCAD,0.000759,Normal
199,3.9,0.000000,0.493199,0.000000,0.422315,0.641605,DSP_JCAD,0.219290,Normal
239,3.9,0.497400,0.000000,0.000000,0.645544,0.412763,TJP1_JCAD,-0.232781,Normal


In [33]:

PROJECT_ROOT = Path("/Users/emilieyu/endotehelial-masboss/")
CONFIG_PATH = PROJECT_ROOT / "config" / "rho_sim_config.yaml"

with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

# Resolve Paths
MODELS_BND = PROJECT_ROOT / config['paths']['model_bnd']
MODELS_CFG = PROJECT_ROOT / config['paths']['model_cfg']
RESULTS_DIR = PROJECT_ROOT / config['paths']['results_base']
PARAM_DIR = RESULTS_DIR / config['paths']['subdirs']['param_sweep']

# Result subdirectories
RAW_DIR = RESULTS_DIR / config['paths']['subdirs']['raw']
PROCESSED_DIR = RESULTS_DIR / config['paths']['subdirs']['processed']
SS_DIR = RESULTS_DIR / config['paths']['subdirs']['steady_state']

# Global variable

EPS = config['analysis']['thresholds']['eps']
PERBS_DICT = config.get('perturbations') # get mutations directly from config
N_RUNS=100

In [34]:
base_model = maboss.load(str(MODELS_BND), str(MODELS_CFG))
sweep_cfg = config['sensitivity_analysis']

In [ ]:
# WT Sensitivity analysis for each parameter
result = []
for param_cfg in sweep_cfg:
    name = param_cfg['parameter']
    print(f"DEBUG: Running sweep for paraneter: {name}")

    values = np.arange(param_cfg['range'][0], param_cfg['range'][1], param_cfg['steps'])

    for val in values: 
        m = base_model.copy()
        m.update_parameters(**{name: val})
        #print(m.param[name])

        res = m.run()
        ss_df = res.get_last_nodes_probtraj()

        ss_df['delta'] = compute_delta(ss_df)
        ss_df['param_value'] = val
        ss_df['param_name'] = name
        

        result.append(ss_df)

combined_df = pd.concat(result, ignore_index=True)
save_df_to_csv(combined_df, PARAM_DIR, "wt_param_sweep_01.csv")


DEBUG: File wt_param_sweep_01.csv successfully written to directory: /Users/emilieyu/endotehelial-masboss/results/boolean_models/param_sweep


In [38]:
test_df = combined_df[combined_df['param_name'].str.contains('amp')]
test_df

,DSP,JCAD,RhoA,RhoC,TJP1,delta,param_value,param_name
0,1.000000,1.000000,0.685030,0.834921,1.000000,0.149891,5.0,$RhoA_amp
1,1.000001,1.000001,0.810077,0.809354,1.000001,-0.000723,10.0,$RhoA_amp
2,1.000001,1.000001,0.865416,0.803142,1.000001,-0.062274,15.0,$RhoA_amp
3,1.000000,1.000000,0.897646,0.791224,1.000000,-0.106422,20.0,$RhoA_amp
4,1.000001,1.000001,0.916708,0.780520,1.000001,-0.136188,25.0,$RhoA_amp
5,1.000000,1.000000,0.926469,0.785913,1.000000,-0.140556,30.0,$RhoA_amp
6,1.000001,1.000001,0.935845,0.786098,1.000001,-0.149747,35.0,$RhoA_amp
7,1.000000,1.000000,0.943901,0.779998,1.000000,-0.163903,40.0,$RhoA_amp
8,1.000000,1.000000,0.950425,0.784661,1.000000,-0.165764,45.0,$RhoA_amp
33,1.000000,1.000000,0.837434,0.683702,1.000000,-0.153732,5.0,$RhoC_amp


In [41]:

test_df['phenotype'] = test_df['delta'].apply(classify_phenotype)
test_df

/var/folders/6g/q6rvxn7j5zz8mmsx5fy77vj80000gn/T/ipykernel_99052/2635949981.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['phenotype'] = test_df['delta'].apply(classify_phenotype)


,DSP,JCAD,RhoA,RhoC,TJP1,delta,param_value,param_name,phenotype
0,1.000000,1.000000,0.685030,0.834921,1.000000,0.149891,5.0,$RhoA_amp,Normal
1,1.000001,1.000001,0.810077,0.809354,1.000001,-0.000723,10.0,$RhoA_amp,Normal
2,1.000001,1.000001,0.865416,0.803142,1.000001,-0.062274,15.0,$RhoA_amp,Normal
3,1.000000,1.000000,0.897646,0.791224,1.000000,-0.106422,20.0,$RhoA_amp,Normal
4,1.000001,1.000001,0.916708,0.780520,1.000001,-0.136188,25.0,$RhoA_amp,Normal
5,1.000000,1.000000,0.926469,0.785913,1.000000,-0.140556,30.0,$RhoA_amp,Normal
6,1.000001,1.000001,0.935845,0.786098,1.000001,-0.149747,35.0,$RhoA_amp,Normal
7,1.000000,1.000000,0.943901,0.779998,1.000000,-0.163903,40.0,$RhoA_amp,Normal
8,1.000000,1.000000,0.950425,0.784661,1.000000,-0.165764,45.0,$RhoA_amp,Normal
33,1.000000,1.000000,0.837434,0.683702,1.000000,-0.153732,5.0,$RhoC_amp,Normal


## Manual 100 Sim Runs

how is it possible that convergence is at the same probabilities for all simulations??

In [50]:
perb_dict = {}
phenotype_dict = {}
#perbs_ss = []

# Run simulations. 
for name, mutation in PERBS_DICT.items():
    print(f"DEBUG: Running scenario: {name}")

    # Create model of KO scenario
    m = base_model.copy()
    for node, state in mutation.items():
        m.mutate(node, state)


    all_runs = []
    for i in range(N_RUNS):
        res = m.run()
        prob_df = res.get_last_nodes_probtraj()
        prob_df["run"] = i
        all_runs.append(prob_df)

    # Combine all runs
    prob_df = pd.concat(all_runs, ignore_index=True)

    # Compute Rho balance
    balance_df = prob_df.copy()
    balance_df["delta"] = compute_delta(balance_df)

    perb_dict[name] = balance_df
    
    # phenotype_df = classify_phenotype(balance_df)
    # phenotype_dict[name] = phenotype_df
    #print(phenotype_df)

DEBUG: Running scenario: WT
DEBUG: Running scenario: DSP
DEBUG: Running scenario: TJP1
DEBUG: Running scenario: JCAD
DEBUG: Running scenario: DSP_JCAD
DEBUG: Running scenario: TJP1_JCAD


In [54]:
dsp = perb_dict['DSP']
dsp['delta'].unique()

array([0.668165])

## Ensemble Moment

In [5]:

res = base_model.run()
base_model.get_initial_state()
ensemble = maboss.Ensemble(str(PROJECT_ROOT / "data" / "models"), str(MODELS_CFG), simulations=100)
ensemble.param['thread_count'] = 1
ensemble.print_cfg()


thread_count = 1;
DSP.is_internal = FALSE;
TJP1.is_internal = FALSE;
JCAD.is_internal = FALSE;
RhoA.is_internal = FALSE;
RhoC.is_internal = FALSE;



In [16]:

e = ensemble.copy()
e.mutate('DSP', 'OFF')

In [19]:
e.mutations
res = e.run()
prob_df = res.get_nodes_probtraj()
prob_df

,DSP,TJP1,RhoA,JCAD,RhoC
0.0,1.000000,1.000000,0.354098,1.000000,0.358173
0.1,1.000001,1.000001,0.691923,1.000001,0.695999
0.2,1.000000,1.000000,0.781404,1.000000,0.781084
0.3,1.000000,1.000000,0.804508,1.000000,0.804743
0.4,1.000000,1.000000,0.804364,1.000000,0.808941
...,...,...,...,...,...
49.5,1.000000,1.000000,0.802588,1.000000,0.810467
49.6,1.000000,1.000000,0.801073,1.000000,0.810740
49.7,1.000000,1.000000,0.802616,1.000000,0.812007
49.8,1.000000,1.000000,0.806200,1.000000,0.809442


In [ ]:
ensemble_dict = {}
ss_dict = {}
for name, mutation in PERBS_DICT.items():
    print(f"DEBUG: Running scenario: {name}")

    # Create model of KO scenario
    e = ensemble.copy()
    for node, state in mutation.items():
        e.mutate(node, state)
        e.

    # Run MaBoSS
    res = e.run()
    prob_df = res.get_nodes_probtraj().rename_axis('t').reset_index()
    
    # Compute Rho balance
    balance_df = prob_df.copy()  
    balance_df["delta"] = compute_delta(balance_df)
    ensemble_dict[name] = balance_df

    

    ss_df = res.get_last_nodes_probtraj()
    ss_dict[name] = ss_df

DEBUG: Running scenario: WT
DEBUG: Running scenario: DSP
thread_count = 1;
DSP.is_internal = FALSE;
TJP1.is_internal = FALSE;
JCAD.is_internal = FALSE;
RhoA.is_internal = FALSE;
RhoC.is_internal = FALSE;

DEBUG: Running scenario: TJP1
thread_count = 1;
DSP.is_internal = FALSE;
TJP1.is_internal = FALSE;
JCAD.is_internal = FALSE;
RhoA.is_internal = FALSE;
RhoC.is_internal = FALSE;

DEBUG: Running scenario: JCAD
thread_count = 1;
DSP.is_internal = FALSE;
TJP1.is_internal = FALSE;
JCAD.is_internal = FALSE;
RhoA.is_internal = FALSE;
RhoC.is_internal = FALSE;

DEBUG: Running scenario: DSP_JCAD
thread_count = 1;
DSP.is_internal = FALSE;
TJP1.is_internal = FALSE;
JCAD.is_internal = FALSE;
RhoA.is_internal = FALSE;
RhoC.is_internal = FALSE;

thread_count = 1;
DSP.is_internal = FALSE;
TJP1.is_internal = FALSE;
JCAD.is_internal = FALSE;
RhoA.is_internal = FALSE;
RhoC.is_internal = FALSE;

DEBUG: Running scenario: TJP1_JCAD
thread_count = 1;
DSP.is_internal = FALSE;
TJP1.is_internal = FALSE;
JCAD.

In [43]:
ensemble_dict['TJP1']

,t,DSP,TJP1,RhoA,JCAD,RhoC,delta
0,0.0,1.000000,1.000000,0.354098,1.000000,0.358173,0.004075
1,0.1,1.000001,1.000001,0.691923,1.000001,0.695999,0.004076
2,0.2,1.000000,1.000000,0.781404,1.000000,0.781084,-0.000320
3,0.3,1.000000,1.000000,0.804508,1.000000,0.804743,0.000235
4,0.4,1.000000,1.000000,0.804364,1.000000,0.808941,0.004577
...,...,...,...,...,...,...,...
495,49.5,1.000000,1.000000,0.802588,1.000000,0.810467,0.007879
496,49.6,1.000000,1.000000,0.801073,1.000000,0.810740,0.009667
497,49.7,1.000000,1.000000,0.802616,1.000000,0.812007,0.009391
498,49.8,1.000000,1.000000,0.806200,1.000000,0.809442,0.003242


In [40]:
ss_dict

{'WT':               DSP      JCAD      RhoA      RhoC      TJP1
 49.9000  0.999999  0.999999  0.806665  0.814914  0.999999,
 'DSP':               DSP      JCAD      RhoA      RhoC      TJP1
 49.9000  0.999999  0.999999  0.806665  0.814914  0.999999,
 'TJP1':               DSP      JCAD      RhoA      RhoC      TJP1
 49.9000  0.999999  0.999999  0.806665  0.814914  0.999999,
 'JCAD':               DSP      JCAD      RhoA      RhoC      TJP1
 49.9000  0.999999  0.999999  0.806665  0.814914  0.999999,
 'DSP_JCAD':               DSP      JCAD      RhoA      RhoC      TJP1
 49.9000  0.999999  0.999999  0.806665  0.814914  0.999999,
 'TJP1_JCAD':               DSP      JCAD      RhoA      RhoC      TJP1
 49.9000  0.999999  0.999999  0.806665  0.814914  0.999999}